# import library

In [5]:
import os
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"
os.environ["XLA_FLAGS"] = "--xla_gpu=false"

import tensorflow as tf
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# load dataset

In [6]:
dataset_path = "garbage_classification"

img_size = 160    # NHỎ lại → ít RAM
batch_size = 8    # NHỎ để tránh crash

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

num_classes = len(train_generator.class_indices)
print("Số lớp:", num_classes)
with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)

Found 8133 images belonging to 12 classes.
Found 2030 images belonging to 12 classes.
Số lớp: 12


# creat model EfficientNetB0

In [7]:
base_model = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(img_size, img_size, 3)
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 80, 80,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 80, 80,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 80, 80,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 80, 80,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 80, 80,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 80, 80,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 81, 81,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 40, 40,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 40, 40,    │      2,304 │ block_1_depthwis

 Total params: 2,273,356 (8.67 MB)

 Trainable params: 2,239,244 (8.54 MB)

 Non-trainable params: 34,112 (133.25 KB)

# call back

In [8]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2)
]


# trainning model

In [9]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20


2025-11-15 11:14:20.883079: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 20155392 exceeds 10% of free system memory.
2025-11-15 11:14:21.157137: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 25497600 exceeds 10% of free system memory.
2025-11-15 11:14:21.157181: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 25497600 exceeds 10% of free system memory.
2025-11-15 11:14:21.167210: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 25497600 exceeds 10% of free system memory.
2025-11-15 11:14:21.167306: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 25497600 exceeds 10% of free system memory.


1017/1017 ━━━━━━━━━━━━━━━━━━━━ 383s 357ms/step - accuracy: 0.5863 - loss: 1.3414 - val_accuracy: 0.1562 - val_loss: 9.0313 - learning_rate: 0.0010
Epoch 2/20
1017/1017 ━━━━━━━━━━━━━━━━━━━━ 359s 353ms/step - accuracy: 0.7166 - loss: 0.8857 - val_accuracy: 0.1837 - val_loss: 10.1307 - learning_rate: 0.0010
Epoch 3/20
1017/1017 ━━━━━━━━━━━━━━━━━━━━ 358s 352ms/step - accuracy: 0.7626 - loss: 0.7499 - val_accuracy: 0.3404 - val_loss: 6.1086 - learning_rate: 0.0010
Epoch 4/20
1017/1017 ━━━━━━━━━━━━━━━━━━━━ 358s 352ms/step - accuracy: 0.7966 - loss: 0.6552 - val_accuracy: 0.4793 - val_loss: 3.6683 - learning_rate: 0.0010
Epoch 5/20
1017/1017 ━━━━━━━━━━━━━━━━━━━━ 356s 350ms/step - accuracy: 0.8089 - loss: 0.5931 - val_accuracy: 0.5498 - val_loss: 3.3677 - learning_rate: 0.0010
Epoch 6/20
1017/1017 ━━━━━━━━━━━━━━━━━━━━ 356s 350ms/step - accuracy: 0.8297 - loss: 0.5399 - val_accuracy: 0.5887 - val_loss: 2.2406 - learning_rate: 0.0010
Epoch 7/20
1017/1017 ━━━━━━━━━━━━━━━━━━━━ 355s 349ms/step - ac

# save model

In [10]:
model.save("garbage_effnetb0.h5")
print("Saved model → garbage_effnetb0.h5")

with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)

print("Saved class labels → class_indices.json")
# --- IGNORE ---

Saved model → garbage_effnetb0.h5
Saved class labels → class_indices.json
